In [81]:
# General Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from selenium import webdriver
import main
import seaborn as sns
import chromedriver_autoinstaller
from datetime import datetime
import matplotlib.pyplot as plt
from mplsoccer import Pitch, VerticalPitch
from matplotlib.colors import to_rgba
import matplotlib as mpl
from collections import defaultdict
import pickle
from PIL import Image
import urllib
from matplotlib.patches import FancyBboxPatch


In [82]:
chromedriver_autoinstaller.install()

import warnings
warnings.filterwarnings('ignore')

## Pull Data

In [187]:
chromedriver_autoinstaller.install()

if __name__ == "__main__":
    driver = webdriver.Chrome()
    
# whoscored match centre url of the required match (Example: Barcelona vs Sevilla)
url = "https://www.whoscored.com/Matches/1640954/Live/England-Premier-League-2022-2023-Wolverhampton-Bournemouth"
match_data = main.getMatchData(driver, url, close_window=True)

# Match dataframe containing info about the match
matches_df = main.createMatchesDF(match_data)

# Events dataframe      
events_df = main.createEventsDF(match_data)

# match Id
matchId = match_data['matchId']

# Information about respective teams as dictionary
home_data = matches_df['home'][matchId]
away_data = matches_df['away'][matchId]
score = matches_df['score'][matchId]
score = ' ' + score.replace(':', '-') + ' '

home_name = home_data['name']
away_name = away_data['name']
date = match_data['startDate'].split('T')[0]
date_ = datetime.strptime(date, '%Y-%m-%d').date()
date_str = date_.strftime('%d/%m/%Y')

Region: England, League: Premier League, Season: 2022/2023, Match Id: 1640954


In [188]:
score = score.replace('*', '')

In [189]:
homeID = home_data['teamId']
awayID = away_data['teamId']

## Colours 

In [190]:
home_colors = {
    'Arsenal': '#E63636',
    'Aston Villa' : '#811331',
    'Brentford' : '#E63636', 
    'Bournemouth' : '#E63636',
    'Brighton': '#2d8ce7',
    'Chelsea' : '#2D5DE7',
    'Crystal Palace' : '#E63636',
    'Everton' : '#2D5DE7', 
    'Fulham' : '#9C9C9C', 
    'Leeds' : '#9C9C9C', 
    'Leicester' : '#2d8ce7',
    'Liverpool' : '#E63636',
    'Nottingham Forest' : '#E63636',
    'Newcastle' : '#9C9C9C',
    'Man Utd' : '#E63636',
    'Man City' : '#6FC6E6',
    'Southampton' : '#E63636',
    'Tottenham' : '#9C9C9C', 
    'West Ham' : '#811331',
    'Wolves' : '#DA9D0A'
}

away_colors = {
    'Arsenal': '#757474',
    'Aston Villa' : '#6FC6E6',
    'Brentford' : '#6FC6E6', 
    'Bournemouth' : '#757474',
    'Brighton': '#FB6B07',
    'Chelsea' : '#C5C5AD',
    'Crystal Palace' : '#9C9C9C', 
    'Everton' : '#C571FA', 
    'Fulham' : '#5FE347', 
    'Leeds' : '#b4b400', 
    'Leicester' : '#757474',
    'Liverpool' : '#9C9C9C', 
    'Nottingham Forest' : '#b4b400',
    'Newcastle' : '#0202C4',
    'Man Utd' : '#9C9C9C',
    'Man City' : '#757474',
    'Southampton' : '#2d8ce7',
    'Tottenham' : '#C571FA', 
    'West Ham' : '#757474',
    'Wolves' : '#066D49'
}

In [191]:
home_color = home_colors[home_name]
away_color = home_colors[away_name]

if (home_color == away_color) or (home_color == '#6FC6E6' and away_color == '#2d8ce7') or (home_color == '#2d8ce7' and away_color == '#6FC6E6') or (home_color == '#E63636' and away_color == '#811331') or (home_color == '#811331' and away_color == '#E63636'):
    away_color = away_colors[away_name]


## Logos|

In [192]:
logos = {
    'Arsenal': 9825,
    'Aston Villa' : 10252,
    'Brentford' : 9937, 
    'Bournemouth' : 8678, 
    'Brighton': 10204,
    'Chelsea' : 8455,
    'Crystal Palace' : 9826, 
    'Everton' : 8668, 
    'Fulham' : 9879, 
    'Leeds' : 8463, 
    'Leicester' : 8197,
    'Liverpool' : 8650,
    'Nottingham Forest' : 10203,
    'Newcastle' : 10261,
    'Man Utd' : 10260,
    'Man City' : 8456,
    'Southampton' : 8466,
    'Tottenham' : 8586, 
    'West Ham' : 8654,
    'Wolves' : 8602
}

home_logo_code = logos[home_name]
away_logo_code = logos[away_name]

## Get 11 players with most minutes

In [193]:
# For the home team
team_players = events_df[events_df.teamId == homeID].groupby('playerName').agg({'minute': [min, max]}).reset_index()
team_players = pd.concat([team_players['playerName'], team_players['minute']], axis=1)
team_players['mins_played'] = team_players['max'] - team_players['min']
team_players = team_players.sort_values('mins_played', ascending=False)
team_players_names = team_players.playerName
team_players_names_top11 = team_players.playerName[:11].tolist()


# For the away team
team2_players = events_df[events_df.teamId == awayID].groupby('playerName').agg({'minute': [min, max]}).reset_index()
team2_players = pd.concat([team2_players['playerName'], team2_players['minute']], axis=1)
team2_players['mins_played'] = team2_players['max'] - team2_players['min']
team2_players = team2_players.sort_values('mins_played', ascending=False)
team2_players_names = team2_players.playerName
team2_players_names_top11 = team2_players.playerName[:11].tolist()


# All Players
team_all_names = team_players_names.tolist()
team2_all_names = team2_players_names.tolist()

all_players = team_all_names + team2_all_names
all_players

['Craig Dawson',
 'José Sá',
 'Maximilian Kilman',
 'Nélson Semedo',
 'Rúben Neves',
 'Pablo Sarabia',
 'Adama Traoré',
 'Matheus Cunha',
 'João Moutinho',
 'Hugo Bueno',
 'Matheus Nunes',
 'Daniel Podence',
 'Rayan Aït-Nouri',
 'Diego Costa',
 'João Gomes',
 'Raúl Jiménez',
 'Jack Stephens',
 'Philip Billing',
 'Dominic Solanke',
 'Marcos Senesi',
 'Neto',
 'Adam Smith',
 'Jordan Zemura',
 'Hamed Traorè',
 'Dango Ouattara',
 'Marcus Tavernier',
 'Jefferson Lerma',
 'Joe Rothwell',
 'Jaidon Anthony',
 'Antoine Semenyo',
 'Chris Mepham',
 'Matías Viña']

## Data Engineering - xA
- Just shifting index by one won't capture actions that should qualify as a potential assist and ergo xA values. 
- E.g. if a player carries the ball/takes a 'ball touch' then the preceding action won't qualify as an assist if we just shift the index by one
- Hence we're going to get all key passes for each player from the events_df dataframe, then just iterate through subsequent indexes until we get the first shot 
- Then just merge the id on the xG df

In [194]:
keyPassCols = ['keyPassLong', 
               'keyPassShort', 
               'keyPassCross',
               'keyPassCorner', 
               'keyPassThroughball', 
               'keyPassFreekick', 
               'keyPassThrowin', 
               'keyPassOther'
              ]

shot_types = ['SavedShot', 
              'MissedShots', 
              'Goal', 
              'ShotOnPost'
             ]

df_KP = pd.DataFrame(columns=['playerAssist', 'id'])

for player in all_players:
    index = []
    for col in keyPassCols:
        idxs = events_df[(events_df['playerName'] == player) &
                         (events_df[col] == 1)].index.values.tolist()
        for idx in idxs:
            if idx not in index:
                index.append(idx)

    # Now need to get correspoding id of subsequent shot, then just merge on the xA dataframe

    for idx in index:
        df = events_df.loc[idx+1:]

        for idx, row in df.iterrows():
            if any(row['type'] == x for x in shot_types):
                df_KP.loc[len(df_KP)] = [player, row['id']]
                break

            

df_KP

,playerAssist,id
0,Nélson Semedo,2.509204e+09
1,Pablo Sarabia,2.509129e+09
2,Pablo Sarabia,2.509311e+09
3,Pablo Sarabia,2.509265e+09
4,Pablo Sarabia,2.509299e+09
5,Adama Traoré,2.509168e+09
6,Adama Traoré,2.509179e+09
7,Adama Traoré,2.509262e+09
8,Adama Traoré,2.509291e+09
9,Matheus Cunha,2.509213e+09


## XG Model
- Run shot data through our xG model, so we can get the xA and xG for individual players (and how they rank relative to distribution)
- First run through function which reduces it to the necessary form (described in detail in our xG model notebook)

In [195]:
pitch_length_x = 100
pitch_length_y = 100

feature_cols = ['id',
                'teamId',
                'prevTeam',
                'x', 
                'y',
                'playerName', 
                'playerAssist',
                'minute', 
                'second',
                'type',
                'prevEvent',
                'isGoal',
                'shotRightFoot',
                'shotLeftFoot',
                'shotHead',
                'shotOpenPlay',
                'shotSetPiece',
                'shotCounter', 
#                 'shotDirectCorner',
                'penaltyScored',
                'penaltyMissed',
                'goalOwn',
#                 'shotLongPass',
#                 'shotShortPass',
#                 'shotCross',
#                 'shotThroughBall',
                'bigChanceScored',
                'bigChanceMissed'
              ]

shot_types = ['SavedShot', 
              'MissedShots', 
              'Goal', 
              'ShotOnPost'
             ]


def shotData(df):
    df["passRecipient"] = df['playerName'].shift(-1)
    df['teamRecipient'] = df['teamId'].shift(-1)
    df['prevEvent'] = df['type'].shift(1)
    df['shotLongPass'] = df['keyPassLong'].shift(1)
    df['shotShortPass'] = df['keyPassShort'].shift(1)
    df['shotCross'] = df['keyPassCross'].shift(1)
    df['shotThroughBall'] = df['keyPassThroughball'].shift(1)
    df['playerAssist'] = df['playerName'].shift(1)
    df['prevTeam'] = df['teamId'].shift(1)

    column_to_move = df.pop("passRecipient")
    df.insert(29, "passRecipient", column_to_move)
    column_to_move = df.pop("teamRecipient")
    df.insert(27, "teamRecipient", column_to_move)
    column_to_move = df.pop('prevEvent')
    df.insert(11, "prevEvent", column_to_move)

    df_shots = df[df['type'].isin(shot_types)].reset_index(drop=True)[feature_cols]

    df_shots = df_shots[(df_shots['penaltyScored'] == False) & 
                        (df_shots['penaltyMissed'] == False) &
                        (df_shots['goalOwn'] == False)
                       ]
#     df_shots['shotTakeOn'] = df_shots['prevEvent'] == 'TakeOn'
#     df_shots['shotDefensiveAction'] = df_shots['prevEvent'].isin(['Challenge', 'Tackle', 'Interception', 'Clearance',
#                                                                   'BallRecovery', 'BlockedPass'])

    df_shots['distance_to_goal'] = np.sqrt(((pitch_length_x - df_shots['x'])**2 + 
                                    (pitch_length_y - df_shots['y'])**2))
    df_shots['distance_to_center'] = abs(df_shots['y'] - pitch_length_y/2)
    df_shots['angle'] = np.absolute(np.degrees(np.arctan((abs((pitch_length_y/2) - df_shots['y'])) 
                                                         / (pitch_length_x - df_shots['x']))))


    df_shots = df_shots.drop(['prevEvent', 'penaltyScored', 'penaltyMissed', 'goalOwn', 'type'], axis=1)
    
    # Add Big Chance Column
    df_shots['bigChance'] = df_shots['bigChanceMissed'] | df_shots['bigChanceScored']
    df_shots = df_shots.drop(['bigChanceMissed', 'bigChanceScored'], axis=1)

    cat_cols = ['shotRightFoot',
            'shotLeftFoot',
            'shotHead',
            'shotOpenPlay',
            'shotSetPiece',
#             'shotDirectCorner',
            'shotCounter',
#             'shotLongPass',
#             'shotShortPass',
#             'shotCross',
#             'shotThroughBall',
#             'shotTakeOn', 
#             'shotDefensiveAction',
            'bigChance'
           ]
    df_shots[cat_cols] *= 1
    
    obj_cols = [
#             'shotLongPass',
#             'shotShortPass',
#             'shotCross',
#             'shotThroughBall',
             ]
    
    for col in obj_cols:
        df_shots[col] = pd.to_numeric(df_shots[col])
    

    return df_shots 

In [196]:
data = shotData(events_df).reset_index(drop=True)
X_game = data.iloc[:, 10:]
Y_game = data['isGoal']
X_game

,shotRightFoot,shotLeftFoot,shotHead,shotOpenPlay,shotSetPiece,shotCounter,distance_to_goal,distance_to_center,angle,bigChance
0,0,0,1,1,0,0,49.320685,1.3,9.462322,0
1,1,0,0,0,1,0,50.388590,0.1,0.818455,0
2,0,1,0,1,0,0,34.984139,15.5,69.484475,0
3,0,1,0,1,0,0,54.918121,4.8,53.130102,0
4,0,0,1,1,0,0,45.016553,5.5,38.966747,0
5,1,0,0,1,0,0,74.629351,22.3,50.321021,0
6,0,0,1,1,0,0,57.623780,7.3,50.117315,0
7,0,1,0,1,0,0,30.383713,20.1,74.962184,0
8,1,0,0,1,0,0,69.282393,18.6,62.457717,0
9,1,0,0,1,0,0,41.685729,15.3,33.518009,0


In [197]:
# Import our xG model 
xg_model = pickle.load(open('xgModel_XGBoost22.sav', 'rb'))

In [198]:
y_pred_game = xg_model.predict_proba(X_game)
df_pred_game = pd.DataFrame(y_pred_game, columns = ['prob_no_goal', 'prob_goal'])
df_pred_game = pd.merge(data, df_pred_game, left_index=True, right_index=True)

shots_to_merge = df_pred_game[['id','playerName', 'prob_goal', 'teamId']]
shots_to_merge = (shots_to_merge.merge(df_KP, how='outer', on='id')
                                .fillna('None')
                                .drop_duplicates('id', keep='first')
                                .reset_index(drop=True)
                )
shots_to_merge                                            

,id,playerName,prob_goal,teamId,playerAssist
0,2.509122e+09,Dominic Solanke,0.032126,183,Hamed Traorè
1,2.509129e+09,Matheus Cunha,0.095435,161,Pablo Sarabia
2,2.509133e+09,Matheus Nunes,0.091841,161,Hugo Bueno
3,2.509168e+09,Pablo Sarabia,0.081431,161,Adama Traoré
4,2.509179e+09,Matheus Cunha,0.078152,161,Adama Traoré
5,2.509190e+09,Matheus Cunha,0.007481,161,João Moutinho
6,2.509204e+09,João Moutinho,0.035169,161,Nélson Semedo
7,2.509208e+09,Pablo Sarabia,0.050503,161,João Moutinho
8,2.509213e+09,Adama Traoré,0.073694,161,Matheus Cunha
9,2.509250e+09,Rúben Neves,0.018579,161,João Moutinho


In [199]:
home_xg = df_pred_game.groupby(['teamId'])['prob_goal'].sum()[homeID]
away_xg = df_pred_game.groupby(['teamId'])['prob_goal'].sum()[awayID]
df_pred_game.groupby(['teamId'])['prob_goal'].sum()

teamId
161    0.877051
183    0.889691
Name: prob_goal, dtype: float32

## Will need to save every metric in a list 

In [200]:
metrics = []

## Need to add Penalties to xG

In [201]:
home_pens = len(events_df[(events_df['teamId'] == homeID) & 
                          ((events_df['penaltyScored'] == True) | (events_df['penaltyMissed'] == True))])
away_pens = len(events_df[(events_df['teamId'] == awayID) & 
                          ((events_df['penaltyScored'] == True) | (events_df['penaltyMissed'] == True))])

home_xg += (0.75*home_pens)
away_xg += (0.75*away_pens)

metrics.append(['xG', round(home_xg,2), round(away_xg,2)])
metrics

[['xG', 0.88, 0.89]]

## Open-Play and Set-Piece xG 

In [202]:
# For Home Team
home_opxg = df_pred_game[(df_pred_game['teamId'] == homeID) &
                         ((df_pred_game['shotOpenPlay'] == 1) | (df_pred_game['shotCounter'] == 1)) & 
                         (df_pred_game['shotSetPiece'] == 0)].reset_index(drop=True)['prob_goal'].sum()

home_spxg = df_pred_game[(df_pred_game['teamId'] == homeID) &
                         (df_pred_game['shotOpenPlay'] == 0) & 
                         (df_pred_game['shotSetPiece'] == 1)].reset_index(drop=True)['prob_goal'].sum()

# For Away Team
away_opxg = df_pred_game[(df_pred_game['teamId'] == awayID) &
                        ((df_pred_game['shotOpenPlay'] == 1) | (df_pred_game['shotCounter'] == 1)) & 
                         (df_pred_game['shotSetPiece'] == 0)].reset_index(drop=True)['prob_goal'].sum()

away_spxg = df_pred_game[(df_pred_game['teamId'] == awayID) &
                         (df_pred_game['shotOpenPlay'] == 0) & 
                         (df_pred_game['shotSetPiece'] == 1)].reset_index(drop=True)['prob_goal'].sum()

metrics.append(['Open Play xG', round(home_opxg, 2), round(away_opxg, 2)])
metrics.append(['Set Piece xG', round(home_spxg, 2), round(away_spxg, 2)])
metrics

[['xG', 0.88, 0.89], ['Open Play xG', 0.65, 0.89], ['Set Piece xG', 0.23, 0.0]]

## Expected Points and Win Prob
- Assuming Poisson distribution here 
- Just running 1,000,000 scenarios and looking at the outcomes

In [203]:
sims = 1000000

home_wins = 0
away_wins = 0
draws = 0

for _ in range(sims):
    home_goals = np.random.poisson(home_xg)
    away_goals = np.random.poisson(away_xg)
    
    if home_goals > away_goals:
        home_wins += 1
        
    if away_goals > home_goals:
        away_wins += 1
        
    if home_goals == away_goals:
        draws += 1
        
        
home_win_prob = home_wins/sims
away_win_prob = away_wins/sims
draw_prob = draws/sims

In [204]:
home_xp = (home_win_prob*3) + (draw_prob*1)
away_xp = (away_win_prob*3) + (draw_prob*1)

In [205]:
home_xp, away_xp

(1.322874, 1.344453)

In [206]:
metrics.append(['Win Probability', str(int(home_win_prob*100))+'%', str(int(away_win_prob*100))+'%'])
metrics

[['xG', 0.88, 0.89],
 ['Open Play xG', 0.65, 0.89],
 ['Set Piece xG', 0.23, 0.0],
 ['Win Probability', '33%', '33%']]

## Shots and Big Chances 

In [207]:
# Shots
home_shots = len(df_pred_game[df_pred_game['teamId'] == homeID]) + home_pens
away_shots = len(df_pred_game[df_pred_game['teamId'] == awayID]) + away_pens
metrics.append(['Shots', home_shots, away_shots])

# # Big Chances
# home_bc = len(df_pred_game[(df_pred_game['teamId'] == homeID) &
#                            (df_pred_game['prob_goal'] > 0.275)]) + home_pens
# away_bc = len(df_pred_game[(df_pred_game['teamId'] == awayID) &
#                            (df_pred_game['prob_goal'] > 0.275)]) + away_pens
# metrics.append(['Big Chances', home_bc, away_bc])

## Possession 

In [208]:
home_poss = sum(matches_df['home'][matchId]['stats']['possession'].values())
away_poss = sum(matches_df['away'][matchId]['stats']['possession'].values())
home_poss_val = home_poss/(home_poss+away_poss)
home_poss_val = round(home_poss_val, 2)*100
away_poss_val = 100-home_poss_val

metrics.append(['Posession', str(int(home_poss_val))+'%', str(int(away_poss_val))+'%'])

## Passes 

In [209]:
passes_h = sum(matches_df['home'][matchId]['stats']['passesTotal'].values())
passes_a = sum(matches_df['away'][matchId]['stats']['passesTotal'].values())
metrics.append(['Passes', int(passes_h), int(passes_a)])

## Corners 

In [210]:
try:
    corners_h = sum(matches_df['home'][matchId]['stats']['cornersTotal'].values())
except KeyError:
    corners_h = 0

try:
    corners_a = sum(matches_df['away'][matchId]['stats']['cornersTotal'].values())
except KeyError:
    corners_a = 0
metrics.append(['Corners', int(corners_h), int(corners_a)])

## Fouls

In [211]:
fouls_h = sum(matches_df['home'][matchId]['stats']['foulsCommited'].values())
fouls_a = sum(matches_df['away'][matchId]['stats']['foulsCommited'].values())
metrics.append(['Fouls', int(fouls_h), int(fouls_a)])

## Field Tilt
- Measures share of possession a team has in terms of passes in attacking 1/3 of the pitch

In [212]:
home_ft_val = len(events_df[(events_df['teamId'] == homeID) &
                            (events_df['type'] == 'Pass') & 
#                             (events_df['outcomeType'] == 'Successful') & 
                            (events_df['x'] > 66.6) & 
                            (events_df['endX'] > 66.6)].reset_index(drop=True))

away_ft_val = len(events_df[(events_df['teamId'] == awayID) &
                            (events_df['type'] == 'Pass') &                           
#                             (events_df['outcomeType'] == 'Successful') & 
                            (events_df['x'] > 66.6) & 
                            (events_df['endX'] > 66.6)].reset_index(drop=True))

den = home_ft_val + away_ft_val
home_ft, away_ft = (home_ft_val/den, away_ft_val/den)
metrics.append(['Field Tilt', str(round(home_ft*100,2))+'%', str(round(away_ft*100,2))+'%'])

## PPDA (Passes Per Defensive Action) 

In [213]:
# Opponent Passes in Defensive 3/5 
passes = len(events_df[(events_df['teamId'] == awayID) & 
                       (events_df['type'] == 'Pass') &
                       (events_df['outcomeType'] == 'Successful') & 
                       (events_df['x'] <= 60)])
defensive_actions = len(events_df[(events_df['teamId'] == homeID) & 
                                  (events_df['type'].isin(['Tackle', 'Challenge', 'Interception', 
                                                           'Foul',])) & 
                                  (events_df['x'] >= 40)])
home_ppda = round(passes/defensive_actions,2)

# For Away Team
passes2 = len(events_df[(events_df['teamId'] == homeID) & 
                       (events_df['type'] == 'Pass') &
                       (events_df['outcomeType'] == 'Successful') & 
                       (events_df['x'] <= 60)])
defensive_actions2 = len(events_df[(events_df['teamId'] == awayID) & 
                                  (events_df['type'].isin(['Tackle', 'Challenge', 'Interception', 
                                                           'Foul',])) & 
                                  (events_df['x'] >= 40)])
away_ppda = round(passes2/defensive_actions2,2)
metrics.append(['PPDA', home_ppda, away_ppda])

## Passing Directness 

In [214]:
df_home_directness = events_df[(events_df['teamId'] == homeID) & 
                               (events_df['type'] == 'Pass') & 
                               (events_df['passCorner'] == False) & 
                               (events_df['passFreekick'] == False)].reset_index(drop=True)[['id', 'x', 'y', 'endX', 'endY']]
df_home_directness['startDist'] = ''
df_home_directness['endDist'] = ''
df_home_directness['ProgPass'] = ''

for i in range(len(df_home_directness)):
    x0 = df_home_directness['x'][i]
    x1 = df_home_directness['endX'][i]
    y0 = df_home_directness['y'][i]
    y1 = df_home_directness['endY'][i]
    
    startDist = np.sqrt((100 - x0)**2 + (50 - y0)**2)
    endDist = np.sqrt((100 - x1)**2 + (50 - y1)**2)
    
    df_home_directness['startDist'][i] = startDist
    df_home_directness['endDist'][i] = endDist
    
    if endDist/startDist <= 0.75:
        df_home_directness['ProgPass'][i] = 1
    else:
        df_home_directness['ProgPass'][i] = 0
    
    
df_home_directness['changeDist'] = df_home_directness['endDist'] - df_home_directness['startDist']

fwd_dist = abs(df_home_directness[df_home_directness['changeDist'] < 0]['changeDist']).sum()
prog_dist = abs(df_home_directness[df_home_directness['ProgPass'] == 1]['changeDist']).sum()
total_dist = abs(df_home_directness['changeDist']).sum()
home_dir = str(round((prog_dist/total_dist)*100, 2))+'%'
home_dir


# For away team
df_away_directness = events_df[(events_df['teamId'] == awayID) & 
                               (events_df['type'] == 'Pass') & 
                               (events_df['passCorner'] == False) & 
                               (events_df['passFreekick'] == False)].reset_index(drop=True)[['id', 'x', 'y', 'endX', 'endY']]
df_away_directness['startDist'] = ''
df_away_directness['endDist'] = ''
df_away_directness['ProgPass'] = ''

for i in range(len(df_away_directness)):
    x0 = df_away_directness['x'][i]
    x1 = df_away_directness['endX'][i]
    y0 = df_away_directness['y'][i]
    y1 = df_away_directness['endY'][i]
    
    startDist = np.sqrt((100 - x0)**2 + (50 - y0)**2)
    endDist = np.sqrt((100 - x1)**2 + (50 - y1)**2)
    
    df_away_directness['startDist'][i] = startDist
    df_away_directness['endDist'][i] = endDist
    
    if endDist/startDist <= 0.75:
        df_away_directness['ProgPass'][i] = 1
    else:
        df_away_directness['ProgPass'][i] = 0
    
df_away_directness['changeDist'] = df_away_directness['endDist'] - df_away_directness['startDist']
fwd_dist = abs(df_away_directness[df_away_directness['changeDist'] < 0]['changeDist']).sum()
prog_dist = abs(df_away_directness[df_away_directness['ProgPass'] == 1]['changeDist']).sum()
total_dist = abs(df_away_directness['changeDist']).sum()
away_dir = str(round((prog_dist/total_dist)*100, 2))+'%'

metrics.append(['Passing Directness', home_dir, away_dir])

In [215]:
metrics

[['xG', 0.88, 0.89],
 ['Open Play xG', 0.65, 0.89],
 ['Set Piece xG', 0.23, 0.0],
 ['Win Probability', '33%', '33%'],
 ['Shots', 15, 5],
 ['Posession', '69%', '31%'],
 ['Passes', 616, 291],
 ['Corners', 12, 2],
 ['Fouls', 12, 10],
 ['Field Tilt', '67.35%', '32.65%'],
 ['PPDA', 5.96, 16.35],
 ['Passing Directness', '41.62%', '53.87%']]

## Shots 

In [216]:
home_shots = df_pred_game[(df_pred_game['teamId'] == homeID)].reset_index(drop=True)[['x', 'y', 'prob_goal']]
away_shots = df_pred_game[(df_pred_game['teamId'] == awayID)].reset_index(drop=True)[['x', 'y', 'prob_goal']]
home_shots['x'] = 100 - home_shots['x']
home_shots['y'] = 100 - home_shots['y']
home_shots

pens_h = events_df[(events_df['teamId'] == homeID) & 
                   ((events_df['penaltyScored'] == True) | (events_df['penaltyMissed'] == True))][['x', 'y']]
pens_h['prob_goal'] = 0.75
pens_h['x'] = 100 - pens_h['x']
pens_h['y'] = 100 - pens_h['y']
home_shots = pd.concat([home_shots, pens_h], axis=0).reset_index(drop=True)

pens_a = events_df[(events_df['teamId'] == awayID) & 
                   ((events_df['penaltyScored'] == True) | (events_df['penaltyMissed'] == True))][['x', 'y']]
pens_a['prob_goal'] = 0.75
away_shots = pd.concat([away_shots, pens_a], axis=0).reset_index(drop=True)

## Highest xG 

In [217]:
shots_all = shots_to_merge[['id','playerName', 'prob_goal', 'teamId']]
shots_all

,id,playerName,prob_goal,teamId
0,2.509122e+09,Dominic Solanke,0.032126,183
1,2.509129e+09,Matheus Cunha,0.095435,161
2,2.509133e+09,Matheus Nunes,0.091841,161
3,2.509168e+09,Pablo Sarabia,0.081431,161
4,2.509179e+09,Matheus Cunha,0.078152,161
5,2.509190e+09,Matheus Cunha,0.007481,161
6,2.509204e+09,João Moutinho,0.035169,161
7,2.509208e+09,Pablo Sarabia,0.050503,161
8,2.509213e+09,Adama Traoré,0.073694,161
9,2.509250e+09,Rúben Neves,0.018579,161


In [218]:
highest_xg_home = pd.DataFrame(shots_all[shots_all['teamId'] == homeID].reset_index(drop=True).groupby(['playerName'])['prob_goal'].sum()).reset_index().sort_values(by='prob_goal', ascending=False).reset_index(drop=True)[:3]
highest_xg_home

,playerName,prob_goal
0,Matheus Cunha,0.394526
1,Pablo Sarabia,0.131934
2,Craig Dawson,0.097361


## Highest xA

In [219]:
shots_assists = shots_to_merge[shots_to_merge['playerAssist'] != 'None'].reset_index(drop=True)
highest_xa_home = pd.DataFrame(shots_assists[shots_assists['teamId'] == homeID].reset_index(drop=True).groupby(['playerAssist'])['prob_goal'].sum()).reset_index().sort_values(by='prob_goal', ascending=False).reset_index(drop=True)[:3]
highest_xa_home

,playerAssist,prob_goal
0,Pablo Sarabia,0.293597
1,Adama Traoré,0.272240
2,Hugo Bueno,0.091841


## Highest Shots 

In [220]:
highest_shots_home = pd.DataFrame(shots_all[shots_all['teamId'] == homeID].reset_index(drop=True).groupby(['playerName'])['prob_goal'].count()).reset_index().sort_values(by='prob_goal', ascending=False).reset_index(drop=True)

pens_h = events_df[(events_df['teamId'] == homeID) & 
                   ((events_df['penaltyScored'] == True) | (events_df['penaltyMissed'] == True))][['playerName', 'x']]
pens_h = pd.DataFrame(pens_h.groupby('playerName')['x'].count()).reset_index()
pens_h.rename({'x':'prob_goal'}, axis='columns', inplace=True)
highest_shots_home = pd.DataFrame(pd.concat([highest_shots_home, pens_h], axis=0).groupby('playerName')['prob_goal'].sum()).reset_index().sort_values(by='prob_goal', ascending=False).reset_index(drop=True)[:3]
highest_shots_home

,playerName,prob_goal
0,Matheus Cunha,6
1,Craig Dawson,2
2,Pablo Sarabia,2


## Repeat for Away Team

In [221]:
highest_xg_away = pd.DataFrame(shots_all[shots_all['teamId'] == awayID].reset_index(drop=True).groupby(['playerName'])['prob_goal'].sum()).reset_index().sort_values(by='prob_goal', ascending=False).reset_index(drop=True)[:3]
highest_xa_away =  pd.DataFrame(shots_assists[shots_assists['teamId'] == awayID].reset_index(drop=True).groupby(['playerAssist'])['prob_goal'].sum()).reset_index().sort_values(by='prob_goal', ascending=False).reset_index(drop=True)[:3]
highest_shots_away = pd.DataFrame(shots_all[shots_all['teamId'] == awayID].reset_index(drop=True).groupby(['playerName'])['prob_goal'].count()).reset_index().sort_values(by='prob_goal', ascending=False).reset_index(drop=True)[:3]
pens_a = events_df[(events_df['teamId'] == awayID) & 
                   ((events_df['penaltyScored'] == True) | (events_df['penaltyMissed'] == True))][['playerName', 'x']]
pens_a = pd.DataFrame(pens_a.groupby('playerName')['x'].count()).reset_index()
pens_a.rename({'x':'prob_goal'}, axis='columns', inplace=True)
highest_shots_away = pd.DataFrame(pd.concat([highest_shots_away, pens_a], axis=0).groupby('playerName')['prob_goal'].sum()).reset_index().sort_values(by='prob_goal', ascending=False).reset_index(drop=True)[:3]
highest_shots_away

,playerName,prob_goal
0,Dominic Solanke,3
1,Hamed Traorè,1
2,Marcus Tavernier,1


## Plots

In [222]:
import matplotlib.pyplot as plt
from mplsoccer import Pitch, VerticalPitch
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patheffects as path_effects
import matplotlib.font_manager as fm
import matplotlib.colors as mcolors
from matplotlib import cm
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 700



pitch = Pitch(pitch_type='opta', pitch_color='#2B2B2B', line_color='#c7d5cc',pad_top=100, pad_left = 55,
              pad_bottom = 5, pad_right = 55, pitch_width=80, pitch_length=120,
              linewidth=0.6)
fig, axs = pitch.grid(figheight=10, title_height=0.08, endnote_space=0, axis=False, 
                      title_space=0, grid_height=0.82, endnote_height=0.05)

fig.set_facecolor("#2B2B2B")




axs['title'].text(0, 0.30, home_name.upper(), color='White',
                  va='center', ha='left', fontsize=16, fontname='Sans Serif')
axs['title'].text(1, 0.30, away_name.upper(), color='White',
                  va='center', ha='right', fontsize=16, fontname='Sans Serif')
axs['title'].text(0.5, 0.30, score, color='White',
                  va='center', ha='center', fontsize=16, fontname='Sans Serif')
# axs['title'].hlines(y=0.48, xmin=0, xmax=1, color='white')

# ax['pitch'].annotate(xy=(50, -10), text= date_str.upper() + '  |  '  + 'CREATED BY Chelsea__Chat' ,color='White',
#                   va='center', ha='center', fontsize=6)


y=194
for metric, home, away in metrics:
    if isinstance(home,int) or isinstance(home,np.float32) or isinstance(home,float):
        home2 = home
        
    if isinstance(away,int) or isinstance(away,np.float32) or isinstance(away,float):
        away2 = away
    
    if isinstance(home,str):
        home2 = home[:]
        home = home.replace('%', '')

    if isinstance(away, str):
        away2 = away[:]
        away = away.replace('%', '')
    
    color_box = ''
    if float(home)>float(away):
        color_box = home_color
    elif float(home)<float(away):
        color_box = away_color
    else:
        color_box = ''
        
    axs['pitch'].annotate(text=metric.upper(), 
                          xytext=(50, y), xy=(20, 40), ha='center', va='center', color='white',
                     fontname = 'Sans Serif', fontsize=10)
    
        
    if color_box == home_color:
        if metric == 'Fouls' or metric == 'PPDA':
            axs['pitch'].annotate(text=str(home2), xytext=(0,y), xy=(60, 40), ha='left', va='center', color='white',
                         fontname = 'Sans Serif', fontsize=10)
            axs['pitch'].annotate(text=str(away2), xytext=(100,y), xy=(60, 40), ha='right', va='center', color='white',
                        bbox=dict(facecolor=away_color, edgecolor='none', boxstyle='round,pad=.3'),
                         fontname = 'Sans Serif', fontsize=10)
        else: 
            axs['pitch'].annotate(text=str(home2), xytext=(0,y), xy=(60, 40), ha='left', va='center', color='white',
                             bbox=dict(facecolor=home_color, edgecolor='none', boxstyle='round,pad=.3'),
                             fontname = 'Sans Serif', fontsize=10)
            axs['pitch'].annotate(text=str(away2), xytext=(100,y), xy=(60, 40), ha='right', va='center', color='white',
                             fontname = 'Sans Serif', fontsize=10)

        
    if color_box == away_color:
        if metric == 'Fouls' or metric == 'PPDA':
            axs['pitch'].annotate(text=str(home2), xytext=(0,y), xy=(60, 40), ha='left', va='center', color='white',
                         bbox=dict(facecolor=home_color, edgecolor='none', boxstyle='round,pad=.3'),
                         fontname = 'Sans Serif', fontsize=10)
            axs['pitch'].annotate(text=str(away2), xytext=(100,y), xy=(60, 40), ha='right', va='center', color='white',
                         fontname = 'Sans Serif', fontsize=10)
        else:   
            axs['pitch'].annotate(text=str(home2), xytext=(0,y), xy=(60, 40), ha='left', va='center', color='white',
                             fontname = 'Sans Serif', fontsize=10)
            axs['pitch'].annotate(text=str(away2), xytext=(100,y), xy=(60, 40), ha='right', va='center', color='white',
                            bbox=dict(facecolor=away_color, edgecolor='none', boxstyle='round,pad=.3'),
                             fontname = 'Sans Serif', fontsize=10)
        
        
    else:
        axs['pitch'].annotate(text=str(home2), xytext=(0,y), xy=(60, 40), ha='left', va='center', color='white',
                         fontname = 'Sans Serif', fontsize=10)
        axs['pitch'].annotate(text=str(away2), xytext=(100,y), xy=(60, 40), ha='right', va='center', color='white',
                         fontname = 'Sans Serif', fontsize=10)
    y -= 8

pitch.scatter(home_shots.x, home_shots.y, s=home_shots.prob_goal*500, marker='s',
                           color=home_color, edgecolor = '#2B2B2B', linewidth=1, ax=axs['pitch'])

pitch.scatter(away_shots.x, away_shots.y, s=away_shots.prob_goal*500, marker='s',
                           color=away_color, edgecolor = '#2B2B2B', linewidth=1, ax=axs['pitch'])

axs['endnote'].text(0.00, 0.5, date_str + '  |  ' + 'CREATED BY @JoeW__32', color='white',
                    va='center', ha='left', fontsize=10, fontname = 'Sans Serif') 

p_bbox = FancyBboxPatch((-29, 93),
                        abs(23.5), abs(7),
                        boxstyle="round,pad=0,rounding_size=0.5",
                        ec="none", fc=home_color,
                        mutation_aspect=4
                        )
axs['pitch'].add_patch(p_bbox)
axs['pitch'].annotate(text='npxG'.upper(), xytext=(-17.5, 96), xy=(60, 40), ha='center', 
                     va='center', color='white',
                     fontname = 'Sans Serif', fontsize=10)
y = 89
i = 0
for p1, cnt in zip(highest_xg_home['playerName'], highest_xg_home['prob_goal']):
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    cnt = round(cnt,2)
    axs['pitch'].annotate(text=p1.upper(), 
                          xytext=(-28, y - 4*i), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 8)
    axs['pitch'].annotate(text=cnt, 
                          xytext=(-6.5, y - 4*i), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 8)
    i+=1

p_bbox = FancyBboxPatch((-29, 63),
                        abs(23.5), abs(7),
                        boxstyle="round,pad=0,rounding_size=0.5",
                        ec="none", fc=home_color,
                        mutation_aspect=4
                        )
axs['pitch'].add_patch(p_bbox)
axs['pitch'].annotate(text='xA'.upper(), xytext=(-17.5, 66), xy=(60, 40), ha='center', 
                     va='center', color='white',
                     fontname = 'Sans Serif', fontsize=10)
y = 59
i = 0
for p1, cnt in zip(highest_xa_home['playerAssist'], highest_xa_home['prob_goal']):
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    cnt = round(cnt,2)
    axs['pitch'].annotate(text=p1.upper(), 
                          xytext=(-28, y - 4*i), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 8)
    axs['pitch'].annotate(text=cnt, 
                          xytext=(-6.5, y - 4*i), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 8)
    i += 1

p_bbox = FancyBboxPatch((-29, 33),
                        abs(23.5), abs(7),
                        boxstyle="round,pad=0,rounding_size=0.5",
                        ec="none", fc=home_color,
                        mutation_aspect=4
                        )
axs['pitch'].add_patch(p_bbox)
axs['pitch'].annotate(text='SHOTS'.upper(), xytext=(-17.5, 36), xy=(60, 40), ha='center', 
                     va='center', color='white',
                     fontname = 'Sans Serif', fontsize=10)
y = 29
i = 0
for p1, cnt in zip(highest_shots_home['playerName'], highest_shots_home['prob_goal']):
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    cnt = round(cnt,2)
    axs['pitch'].annotate(text=p1.upper(), 
                          xytext=(-28, y - 4*i), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 8)
    axs['pitch'].annotate(text=cnt, 
                          xytext=(-6.5, y - 4*i), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 8)
    i += 1

axs['pitch'].hlines(y=200, xmin=0, xmax=100, color='white')

# Away
fotmob_url = 'https://images.fotmob.com/image_resources/logo/teamlogo/'
team_logos = away_logo_code
club_icon = Image.open(urllib.request.urlopen(f'{fotmob_url}{team_logos:.0f}.png'))
newax = fig.add_axes([0.815, 0.75, 0.135, 0.135], anchor='NE', zorder=2)
newax.imshow(club_icon)
newax.axis('off')

# Home
team_logos2 = home_logo_code
club_icon2 = Image.open(urllib.request.urlopen(f'{fotmob_url}{team_logos2:.0f}.png'))
newax2 = fig.add_axes([0.04, 0.75, 0.135, 0.135], anchor='NE', zorder=2)
newax2.imshow(club_icon2)
newax2.axis('off')

p_bbox = FancyBboxPatch((106, 93),
                        abs(23.5), abs(7),
                        boxstyle="round,pad=0,rounding_size=0.5",
                        ec="none", fc=away_color,
                        mutation_aspect=4
                        )
axs['pitch'].add_patch(p_bbox)
axs['pitch'].annotate(text='npxG'.upper(), xytext=(117.5, 96), xy=(60, 40), ha='center', 
                     va='center', color='white',
                     fontname = 'Sans Serif', fontsize=10)
y = 89
i = 0
for p1, cnt in zip(highest_xg_away['playerName'], highest_xg_away['prob_goal']):
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    cnt = round(cnt,2)
    axs['pitch'].annotate(text=p1.upper(), 
                          xytext=(107, y - 4*i), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 8)
    axs['pitch'].annotate(text=cnt, 
                          xytext=(128.5, y - 4*i), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 8)
    i+=1
    
p_bbox = FancyBboxPatch((106, 63),
                        abs(23.5), abs(7),
                        boxstyle="round,pad=0,rounding_size=0.5",
                        ec="none", fc=away_color,
                        mutation_aspect=4
                        )
axs['pitch'].add_patch(p_bbox)
axs['pitch'].annotate(text='xA'.upper(), xytext=(117.5, 66), xy=(60, 40), ha='center', 
                     va='center', color='white',
                     fontname = 'Sans Serif', fontsize=10)
y = 59
i = 0
for p1, cnt in zip(highest_xa_away['playerAssist'], highest_xa_away['prob_goal']):
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    cnt = round(cnt,2)
    axs['pitch'].annotate(text=p1.upper(), 
                          xytext=(107, y - 4*i), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 8)
    axs['pitch'].annotate(text=cnt, 
                          xytext=(128.5, y - 4*i), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 8)
    i += 1

p_bbox = FancyBboxPatch((106, 33),
                        abs(23.5), abs(7),
                        boxstyle="round,pad=0,rounding_size=0.5",
                        ec="none", fc=away_color,
                        mutation_aspect=4
                        )
axs['pitch'].add_patch(p_bbox)
axs['pitch'].annotate(text='SHOTS'.upper(), xytext=(117.5, 36), xy=(60, 40), ha='center', 
                     va='center', color='white',
                     fontname = 'Sans Serif', fontsize=10)
y = 29
i = 0
for p1, cnt in zip(highest_shots_away['playerName'], highest_shots_away['prob_goal']):
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    cnt = round(cnt,2)
    axs['pitch'].annotate(text=p1.upper(), 
                          xytext=(107, y - 4*i), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 8)
    axs['pitch'].annotate(text=cnt, 
                          xytext=(128.5, y - 4*i), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 8)
    i += 1

fig.savefig('matchDashboard.png', 
            bbox_inches="tight",
            edgecolor="none",
            dpi=700)